<a href="https://colab.research.google.com/github/siyeol/Natural-Language-Processing/blob/main/Scikit-Learn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Software Coaching for Python
# Week 5: Document Clustering & Topic Modeling - Homework

Instructor: Kang-Pyo Lee

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')
path = "/content/gdrive/Shareddrives/SW_Coaching_for_Python"
my_folder = "SYC"     # *** REPLACE WITH YOUR FOLDER NAME ***
outcome_folder = f"{path}/{my_folder}/outcome"
classdata_folder = f"{path}/classdata"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
! pip install --user scikit-learn pyldavis

     |████████████████████████████████| 1.7MB 6.6MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 9.9MB 22.0MB/s 
  Created wheel for pyldavis: filename=pyLDAvis-3.3.1-cp37-none-any.whl size=136870 sha256=37de2c06c02f39dc0073e576dd7fcc2659cb481cc93ff1da31285eb6c79ea634
  Stored in directory: /root/.cache/pip/wheels/a0/9c/fc/c6e00689d35c82cf96a8adc70edfe7ba7904374fdac3240ac2
Successfully built pyldavis
ERROR: google-colab 1.0.0 has requirement pandas~=1.1.0; python_version >= "3.0", but you'll have pandas 1.2.4 which is incompatible.
ERROR: pyldavis 3.3.1 has requirement numpy>=1.20.0, but you'll have numpy 1.19.5 which is incompatible.


In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', 150)

df = pd.read_csv(f"{classdata_folder}/timeline_NASA.csv", sep="\t")
df.text = df.text.astype(str)
df

,status_id,created_at,text,retweet_count,favorite_count,is_retweet,retweet_created_at
0,1354552906175934477,Wed Jan 27 22:12:33 +0000 2021,"It once held a lake. And what we're going to do, as soon as we get there, is to explore the rocks that were deposi… https://t.co/UW9wngvx4C",72,462,0,NaN
1,1354549784061865986,Wed Jan 27 22:00:09 +0000 2021,"I asked the team [...] to come up with something that would symbolize, to mark this challenge and thank in particu… https://t.co/QH235r9zjq",50,361,0,NaN
2,1354546326139265030,Wed Jan 27 21:46:25 +0000 2021,"Perseverance is the very first rover designed to seek signs of past microbial life, by collecting and caching rock… https://t.co/nn72OHRVv6",135,870,0,NaN
3,1354542309988327425,Wed Jan 27 21:30:27 +0000 2021,"🔴 LIVE: Tune in for a preview of the next Mars landing! On Feb. 18, our @NASAPersevere rover will arrive at the Red… https://t.co/rHRhIrzXWR",374,1430,0,NaN
4,1354540246436872195,Wed Jan 27 21:22:15 +0000 2021,RT @NASAMars: Don't forget to join us for today's briefing at 4:30pm ET (21:30 UTC): https://t.co/fGItMYWnFR Tag your questions #Countdown…,137,0,1,Wed Jan 27 20:18:56 +0000 2021
...,...,...,...,...,...,...,...
3195,1265370733930307585,Tue May 26 19:54:27 +0000 2020,RT @NASAKennedy: Looking for live views of the launch pad? We got you covered. 🚀👀Check out the rocket that will carry @AstroBehnken and @…,1197,0,1,Tue May 26 19:36:00 +0000 2020
3196,1265365848098816002,Tue May 26 19:35:02 +0000 2020,"For the first time since 2011, we’re sending American astronauts back to space, on an American rocket, from America… https://t.co/FsC1feO84c",622,2614,0,NaN
3197,1265327291468562432,Tue May 26 17:01:49 +0000 2020,LIVE: Want to go behind-the-scenes of #LaunchAmerica?Join our experts on @reddit to ask questions about the missi… https://t.co/p7iqJY4jyX,253,1942,0,NaN
3198,1265327258274840576,Tue May 26 17:01:41 +0000 2020,@brandonleblanc @SpaceX @AmyShiraTeitel See you online! https://t.co/FaQ7K75sBP,2,6,0,NaN


Check if there are any duplicates in the `text` column.

In [ ]:
df.text.value_counts()

RT @NASAExhibit: 🤳Share your excitement about space exploration by posting a selfie on Twitter &amp; Instagram tagged with #NASAVirtualExhibit!…    2
@ArtStudioAW 🤩 It's almost as if you can touch them. So realistic! What an amazing way to congratulate Bob and Doug… https://t.co/m0nphsFDo0        1
RT @NASA_Technology: ✔️Laser sensors✔️Camera✔️High-speed computer✔️Sophisticated algorithms Together, these technologies can provide s…             1
@AgnelliMaura @NASAKennedy @Space_Station Uplifting is the word! Crew-1 for all! 🥳🚀 https://t.co/cq6QOQJ8Lq                                         1
What do the Crew-1 astronauts plan to eat right before they launch to the @Space_Station? 🤔A lot. https://t.co/eaOZ2B6QMM                           1
                                                                                                                                                   ..
RT @NASA_Astronauts: Breaking: Butch Wilmore to fly on the Boeing Starliner crew flight test alongsi

Remove duplicates in the `text` column, if any. 

In [ ]:
df = df.drop_duplicates(["text"], keep="first").copy()

In [ ]:
df.text.value_counts()

Lisa Pratt, NASA’s Planetary Protection Officer, is making sure the Mars 2020 Perseverance rover is not transportin… https://t.co/zpVinsoARo    1
Good evening! Konbanwa! @Astro_Soichi of @JAXA_en gives a tour of the lower area of the Crew Dragon Resilience sp… https://t.co/fXFRSjrbNp      1
RT @NASA_Technology: ✔️Laser sensors✔️Camera✔️High-speed computer✔️Sophisticated algorithms Together, these technologies can provide s…         1
@AgnelliMaura @NASAKennedy @Space_Station Uplifting is the word! Crew-1 for all! 🥳🚀 https://t.co/cq6QOQJ8Lq                                     1
What do the Crew-1 astronauts plan to eat right before they launch to the @Space_Station? 🤔A lot. https://t.co/eaOZ2B6QMM                       1
                                                                                                                                               ..
RT @NASA_Astronauts: Breaking: Butch Wilmore to fly on the Boeing Starliner crew flight test alongside NASA Astronauts Nicol

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3199 entries, 0 to 3199
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   status_id           3199 non-null   int64 
 1   created_at          3199 non-null   object
 2   text                3199 non-null   object
 3   retweet_count       3199 non-null   int64 
 4   favorite_count      3199 non-null   int64 
 5   is_retweet          3199 non-null   int64 
 6   retweet_created_at  815 non-null    object
dtypes: int64(4), object(3)
memory usage: 199.9+ KB


## Preparing Data for Modeling

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download(['punkt', 'averaged_perceptron_tagger', 'stopwords'])

import string 
global_stopwords = stopwords.words("english")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Define your own local stopwords. 

In [ ]:
local_stopwords = [c for c in string.punctuation] +\
                  ['‘', '’', '—', '…', '“', '”', "'re", "'s", '...', 'a…', 't…', 'https', "n't", 'rt']

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(use_idf=True, norm="l2", stop_words=global_stopwords+local_stopwords, max_df=0.7)
X = vectorizer.fit_transform(df.text)

In [ ]:
X.shape

(3199, 8679)

## K-Means Clustering

### Step 1. Choose the number of clusters

Set the number of clusters you would like to get. 

In [ ]:
# Your answer here
k = 5

### Step 2. Initialize a model object for k-means clustering

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=k, random_state=0)
kmeans

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=5, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=0, tol=0.0001, verbose=0)

### Step 3. Fit the model using the input data

In [ ]:
%time kmeans.fit(X)

CPU times: user 19.3 s, sys: 15 s, total: 34.2 s
Wall time: 17.4 s


KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=5, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=0, tol=0.0001, verbose=0)

### Step 4. Examine the clustering outcome

In [ ]:
kmeans.labels_

array([1, 1, 0, ..., 1, 1, 2], dtype=int32)

In [ ]:
df["cluster"] = kmeans.labels_

In [ ]:
df[["text", "cluster"]]

,text,cluster
0,"It once held a lake. And what we're going to do, as soon as we get there, is to explore the rocks that were deposi… https://t.co/UW9wngvx4C",1
1,"I asked the team [...] to come up with something that would symbolize, to mark this challenge and thank in particu… https://t.co/QH235r9zjq",1
2,"Perseverance is the very first rover designed to seek signs of past microbial life, by collecting and caching rock… https://t.co/nn72OHRVv6",0
3,"🔴 LIVE: Tune in for a preview of the next Mars landing! On Feb. 18, our @NASAPersevere rover will arrive at the Red… https://t.co/rHRhIrzXWR",0
4,RT @NASAMars: Don't forget to join us for today's briefing at 4:30pm ET (21:30 UTC): https://t.co/fGItMYWnFR Tag your questions #Countdown…,1
...,...,...
3195,RT @NASAKennedy: Looking for live views of the launch pad? We got you covered. 🚀👀Check out the rocket that will carry @AstroBehnken and @…,1
3196,"For the first time since 2011, we’re sending American astronauts back to space, on an American rocket, from America… https://t.co/FsC1feO84c",1
3197,LIVE: Want to go behind-the-scenes of #LaunchAmerica?Join our experts on @reddit to ask questions about the missi… https://t.co/p7iqJY4jyX,1
3198,@brandonleblanc @SpaceX @AmyShiraTeitel See you online! https://t.co/FaQ7K75sBP,1


Check the number of values for each cluster. 

In [ ]:
df.cluster.value_counts()

1    2110
2     383
0     356
3     183
4     167
Name: cluster, dtype: int64

Check 10 random texts from the largest cluster. 

In [ ]:
df[df.cluster == 1].sample(10, random_state=0)[["text", "cluster"]]

,text,cluster
2293,"Scientists are using our @NASAHubble Space Telescope in ways they didn't imagine when it was launched 30 years ago,… https://t.co/pCdwzw0nWL",1
261,RT @nasahqphoto: Take a look back at 100 images highlighting the 2020 work of the NASA Headquarters photo team! 📷 https://t.co/ZjqyGxpOr7 h…,1
2082,"I could not be more proud and more humbled by the courage and dedication of my launch team.Tory Bruno, Presiden… https://t.co/bd0Sqg4Pc9",1
280,"❄️ Newborn stars, hidden behind thick dust, are revealed in this image of the Christmas Tree Cluster. Infant stars… https://t.co/Xetdvb20h6",1
2530,"RT @NASAMoon: A full Moon lights up the night this Independence Day. Called the Buck, Hay, or Thunder⛈️ Moon, this full Moon is also a pe…",1
302,"2020 was historic for NASA. We launched humans to the @Space_Station from the U.S., made progress on plans to retur… https://t.co/j3kJO6HDke",1
1681,RT @KathyLueders: #ICYMI – Robyn Gatens has been named acting director of the @Space_Station. She brings 35 years of experience at @NASA in…,1
878,"Happening now on @Reddit: our astronaut trainers, mission managers, and education specialists join @FAANews to answ… https://t.co/IKwtji9L2v",1
976,"RT @Space_Station: 241 people from around the world have lived and worked on the space station since Nov. 2, 2000. #SpaceStation20th | http…",1
3195,RT @NASAKennedy: Looking for live views of the launch pad? We got you covered. 🚀👀Check out the rocket that will carry @AstroBehnken and @…,1


Check 10 random texts from the second largest cluster. 

In [ ]:
df[df.cluster == 2].sample(10, random_state=0)[["text", "cluster"]]

,text,cluster
1356,RT @NASAKennedy: Four astronauts are getting ready to launch to the @Space_Station on Oct. 31 aboard a @SpaceX Crew Dragon. 🚀 Learn more…,2
768,"Now that the Crew Dragon Resilience spacecraft is safely in orbit, watch continuous coverage of the crew's journey… https://t.co/TZvvWLgwkD",2
1017,LIVE: Watch with us as the Expedition 1 crew — the very first mission to the @Space_Station — sits down with the… https://t.co/VWu0qRHSwt,2
491,“It’s the first time we will have a couple of Dragons on board – a Crew Dragon and a Cargo Dragon.” Hear about… https://t.co/4I1u9GNbjW,2
1528,LIVE: What @ISS_Research is launching from @NASA_Wallops on the next resupply mission to @Space_Station?… https://t.co/0b3TFP6K3v,2
2854,RT @nasahqphoto: .@NASA and @SpaceX teams monitored the docking of #CrewDragon to @Space_Station from firing room four at @NASAKennedy this…,2
2066,"Today, @NASA_Astronauts aboard @Space_Station answer questions about the upcoming departure of the @SpaceX Dragon E… https://t.co/nvvGIrgdvi",2
939,"On Nov. 14, four space travelers launch on a mission to the @Space_Station and you can have a front row seat to the… https://t.co/dQ5MlxOgI4",2
1929,RT @JimBridenstine: UPDATE: Post-SpaceX Crew Dragon DM-2 Splashdown News Conference no earlier than 4:45pm ET. Watch live: https://t.co/qPe…,2
1988,RT @Space_Station: Two very small engine burns separated @SpaceX Endeavour from the station. The spacecraft is slowly maneuvering away from…,2


In [ ]:
import nltk
df["words"] = df.text.apply(lambda x: nltk.word_tokenize(x))
df["tagged_words"] = df.words.apply(lambda x: nltk.pos_tag(x))

from collections import Counter

def get_counter(dataframe, stopwords=[]):
    counter = Counter()
    
    for l in dataframe.tagged_words:
        word_set = set()

        for t in l:
            word = t[0].lower()
            tag = t[1]

            if word not in stopwords:
                word_set.add(word)
            
        counter.update(word_set)
        
    return counter

Check the top-30 most common keywords in the largest cluster. (Replace the cluster number as needed.)

In [ ]:
counter0 = get_counter(df[df.cluster == 1], global_stopwords+local_stopwords)
counter0.most_common(30)

[('space', 199),
 ('nasa', 186),
 ('space_station', 185),
 ('moon', 150),
 ('amp', 142),
 ('today', 139),
 ('live', 131),
 ('us', 128),
 ('launch', 122),
 ('jimbridenstine', 120),
 ('new', 111),
 ('earth', 110),
 ('mission', 89),
 ('launchamerica', 88),
 ('first', 81),
 ('artemis', 80),
 ('nasa_astronauts', 77),
 ('🚀', 77),
 ('years', 77),
 ('et', 73),
 ('one', 70),
 ('rocket', 69),
 ('astronauts', 62),
 ('spacex', 62),
 ('join', 61),
 ('time', 61),
 ('nasakennedy', 58),
 ('thank', 57),
 ('science', 56),
 ('thanks', 54)]

Check the top-30 most common keywords in the second largest cluster. (Replace the cluster number as needed.)

In [ ]:
counter4 = get_counter(df[df.cluster == 2], global_stopwords+local_stopwords)
counter4.most_common(30)

[('spacex', 232),
 ('space_station', 152),
 ('dragon', 127),
 ('crew', 124),
 ('launch', 98),
 ('mission', 94),
 ('crew-1', 86),
 ('nasa', 83),
 ('spacecraft', 75),
 ('cargo', 57),
 ('launchamerica', 48),
 ('nasakennedy', 44),
 ('live', 41),
 ('et', 40),
 ('astronauts', 39),
 ('today', 36),
 ('northropgrumman', 32),
 ('next', 30),
 ('🚀', 30),
 ('resupply', 29),
 ('first', 29),
 ('the…', 28),
 ('cygnus', 27),
 ('rocket', 27),
 ('endeavour', 26),
 ('week', 24),
 ('falcon', 23),
 ('9', 23),
 ('liftoff', 20),
 ('aboard', 20)]

## LDA Topic Modeling

### Step 1. Choose the number of topics

Set the number of topics you would like to get. 

In [ ]:
# Your answer here
num_topics = 5

### Step 2. Initialize a model object for LDA topic modeling

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation as LDA

lda = LDA(n_components=num_topics, random_state=0)
lda

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

### Step 3. Fit the model using the input data

In [ ]:
%time lda.fit(X)

CPU times: user 4.47 s, sys: 45.9 ms, total: 4.51 s
Wall time: 4.49 s


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

### Step 4. Examine the output of topic modeling

In [ ]:
def show_topics(model, feature_names, num_top_words):
    for topic_idx, topic_scores in enumerate(model.components_):
        print("***Topic {}:".format(topic_idx))
        print(" + ".join(["{:.2f} * {}".format(topic_scores[i], feature_names[i]) for i in topic_scores.argsort()[::-1][:num_top_words]]))
        print()

In [ ]:
show_topics(lda, vectorizer.get_feature_names(), 10)

***Topic 0:
14.08 * space_station + 11.37 * crew + 9.76 * northropgrumman + 9.63 * spacex + 8.57 * dragon + 8.39 * spacecraft + 8.31 * us + 8.05 * space + 7.92 * mars + 7.81 * cygnus

***Topic 1:
17.48 * launch + 13.99 * nasa + 13.97 * et + 13.42 * today + 13.41 * sentinel + 12.78 * freilich + 12.70 * michael + 12.56 * launchamerica + 12.42 * live + 11.26 * space_station

***Topic 2:
16.86 * spacex + 15.44 * space_station + 12.03 * astrobehnken + 11.74 * launch + 11.23 * crew + 11.12 * amp + 9.48 * spacecraft + 9.36 * astro_doug + 8.95 * dragon + 8.81 * new

***Topic 3:
19.30 * space_station + 16.42 * astrobehnken + 15.91 * spacex + 15.71 * today + 14.10 * live + 13.89 * nasa + 12.74 * nasapersevere + 12.70 * launch + 12.53 * astronauts + 12.34 * space

***Topic 4:
13.13 * crew + 12.91 * nasa + 11.44 * space_station + 11.09 * mission + 10.82 * spacex + 9.38 * jimbridenstine + 9.08 * moon + 8.13 * launchamerica + 8.06 * astrobehnken + 7.83 * live



### Topic Model Visualization

In [ ]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping


In [ ]:
pyLDAvis.sklearn.prepare(lda, X, vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.046055  0.038682       1        1  23.978385
1     -0.051722  0.032989       2        1  21.233983
0      0.008829 -0.019952       3        1  18.448776
4     -0.003638 -0.030505       4        1  18.267546
2      0.000476 -0.021214       5        1  18.071311, topic_info=                 Term      Freq      Total Category  logprob  loglift
6782         sentinel  7.000000   7.000000  Default  30.0000  30.0000
3154         freilich  7.000000   7.000000  Default  29.0000  29.0000
4904          michael  7.000000   7.000000  Default  28.0000  28.0000
5345  northropgrumman  7.000000   7.000000  Default  27.0000  27.0000
2110           cygnus  5.000000   5.000000  Default  26.0000  26.0000
...               ...       ...        ...      ...      ...      ...
4596             live  3.238514  24.533900   Topic5  -6.3334  -0.3141
5181    nasapersevere  2.979586  19.407679   Topic5  -6.4168  -0.1630
5022             moon  2.895235  18.009074   Topic5  -6.4455  -0.1170
1073       astronauts  2.735800  14.864158   Topic5  -6.5021   0.0183
7624            today  2.724539  25.987621   Topic5  -6.5062  -0.5445

[380 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
67        1  0.226812     12
67        2  0.680435     12
67        5  0.226812     12
85        2  0.624639   15pm
90        2  0.536972     17
...     ...       ...    ...
8497      1  0.189426  years
8497      2  0.284139  years
8497      3  0.284139  years
8497      4  0.094713  years
8497      5  0.094713  years

[615 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 2, 1, 5, 3])